[![Roboflow Notebooks](https://media.roboflow.com/notebooks/template/bannertest2-2.png?ik-sdk-version=javascript-1.4.3&updatedAt=1672932710194)](https://github.com/roboflow/notebooks)

# Train Soccer Player Detector

---

[![Download Dataset](https://app.roboflow.com/images/download-dataset-badge.svg)](https://universe.roboflow.com/roboflow-jvuqo/football-players-detection-3zvbc)

This notebook is based on the [How to Train YOLOv8 Object Detection on a Custom Dataset](https://github.com/roboflow/notebooks/blob/main/notebooks/train-yolov8-object-detection-on-custom-dataset.ipynb) notebook from the [roboflow/notebooks](https://github.com/roboflow/notebooks) repository.

In [1]:
# =================================================================================================
# هذا الملف هو نسخة معدلة من 'LAST_new_train_player_detector.ipynb'
# وهو مصمم للتشغيل في بيئة Kaggle Notebook.
# =================================================================================================

##  إعداد بيئة Kaggle Notebook وتفعيل GPU
#  ------------------------------------------------

#  تأكد من تفعيل "GPU T4 x2" في إعدادات دفتر الملاحظات (Settings -> Accelerator).

#  استيراد المكتبات الأساسية
import os
import shutil
from IPython.display import Image
from kaggle_secrets import UserSecretsClient

#  إعداد المسارات
#  HOME هو المسار الجذر لدفتر الملاحظات في Kaggle
HOME = os.getcwd()
print(f"المسار الحالي: {HOME}")

#  تثبيت المكتبات
#  نستخدم -q لتقليل مخرجات التثبيت
!pip install -q ultralytics roboflow

#  التحقق من توفر GPU
#  سيقوم هذا الأمر بعرض معلومات عن وحدات معالجة الرسومات المتاحة.
print("جارٍ التحقق من حالة GPU...")
!nvidia-smi


المسار الحالي: /kaggle/working
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.6/88.6 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 36.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 104.0 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 91.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 55.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:

#  ================================================================================================

##  تجهيز البيانات والنموذج
#  ------------------------------------------------

#  التعامل مع المفتاح السري Roboflow API
#  يتم استدعاء المفتاح من "الأسرار" (Secrets) في Kaggle
#  تأكد من أنك قمت بحفظ مفتاحك السري باسم "ROBOFLOW_API_KEY"
try:
    user_secrets = UserSecretsClient()
    ROBOFLOW_API_KEY = user_secrets.get_secret("ROBOFLOW_API_KEY")
except Exception as e:
    print(f"خطأ في استدعاء المفتاح السري: {e}")
    ROBOFLOW_API_KEY = None
    
from roboflow import Roboflow

if ROBOFLOW_API_KEY:
    rf = Roboflow(api_key=ROBOFLOW_API_KEY)
    
    #  تحديد مسار جديد للبيانات داخل مجلد عمل Kaggle
    data_folder = os.path.join(HOME, "datasets")
    if not os.path.exists(data_folder):
        os.makedirs(data_folder)
        
    os.chdir(data_folder)
    
    #  تحميل مجموعة البيانات
    project = rf.workspace("shootaai").project("soccer_detection")
    version = project.version(3)
    dataset = version.download("yolov8")
    
    print(f"تم تحميل مجموعة البيانات إلى: {dataset.location}")
    
    #  تعديل ملف data.yaml ليتوافق مع المسارات في Kaggle
    data_yaml_path = os.path.join(dataset.location, 'data.yaml')
    with open(data_yaml_path, 'r') as f:
        lines = f.readlines()
        
    with open(data_yaml_path, 'w') as f:
        for line in lines:
            if line.strip().startswith('train:'):
                f.write('train: ../train/images\n')
            elif line.strip().startswith('val:'):
                f.write('val: ../valid/images\n')
            else:
                f.write(line)
                
    print("تم تحديث مسارات ملف data.yaml")
    os.chdir(HOME)
else:
    print("لم يتم العثور على مفتاح ROBOFLOW_API_KEY. يرجى إضافته في Kaggle Secrets.")


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to soccer_detection-3 in yolov8:: 100%|██████████| 4290/4290 [00:00<00:00, 7621.25it/s]


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
تم تحميل مجموعة البيانات إلى: /kaggle/working/datasets/soccer_detection-3
تم تحديث مسارات ملف data.yaml


In [4]:

#  ================================================================================================

##  بدء عملية التدريب
#  ------------------------------------------------

#  الأمر المعدل للتدريب على وحدتي GPU
#  device=0,1: يحدد استخدام وحدتي GPU
#  batch=20: يزيد من حجم الدفعة للاستفادة الكاملة من موارد الـ GPU
#  epochs=500: عدد تكرارات التدريب
#  imgsz=1280: حجم الصورة
#  plots=True: لإنشاء رسوم بيانية للنتائج
#  model=yolov8x.pt: استخدام نموذج YOLOv8x كنقطة بداية للتدريب
print("بدء تدريب النموذج...")
!yolo task=detect mode=train model=yolov8x.pt data={data_folder}/soccer_detection-3/data.yaml batch=20 epochs=500 imgsz=640 plots=True device=0,1

#  ================================================================================================

##  تقييم النموذج وعرض النتائج
#  ------------------------------------------------

#  التحقق من أداء النموذج بعد التدريب
#  يتم استخدام أفضل وزن (best.pt) للتقييم
print("\nجارٍ تقييم النموذج...")
!yolo task=detect mode=val model={HOME}/runs/detect/train/weights/best.pt data={data_folder}/soccer_detection-3/data.yaml imgsz=640

#  عرض مصفوفة الارتباك (Confusion Matrix)
print("\nمصفوفة الارتباك:")
Image(filename=f'{HOME}/runs/detect/train/confusion_matrix.png', width=600)

#  عرض النتائج الرسومية
print("\nالنتائج الرسومية:")
Image(filename=f'{HOME}/runs/detect/train/results.png', width=600)

#  عرض أمثلة من التنبؤات على دفعة التحقق (Validation Batch)
print("\nأمثلة من التنبؤات:")
Image(filename=f'{HOME}/runs/detect/train/val_batch0_pred.jpg', width=600)

#  ================================================================================================

##  استخراج النموذج المدرب
#  ------------------------------------------------

#  تم حفظ أفضل نموذج في المسار التالي
model_path = os.path.join(HOME, "runs/detect/train/weights/best.pt")
print(f"تم حفظ النموذج المدرب (best.pt) في المسار: {model_path}")

#  لتحميل النموذج، يمكنك استخدام واجهة Kaggle Notebook File Browser (المتصفح الجانبي)
#  أو تشغيل أمر مثل:
#  !cp {model_path} /kaggle/working/best_yolov8x_player_detector.pt

بدء تدريب النموذج...
Ultralytics 8.3.196 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
                                                        CUDA:1 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=20, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/datasets/soccer_detection-3/data.yaml, degrees=0.0, deterministic=True, device=0,1, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=500, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8x.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train2, 

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/working/runs/detect/train/confusion_matrix.png'

In [6]:
!cp /kaggle/working/runs/detect/train2/weights/best.pt /kaggle/working/best.pt
!cp /kaggle/working/runs/detect/train2/weights/last.pt /kaggle/working/last.pt

In [8]:
!pip install -q pydrive2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 36.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 4.6 MB/s eta 0:00:00


In [9]:
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive

gauth = GoogleAuth()
gauth.LocalWebserverAuth() # سيفتح متصفح للمصادقة

drive = GoogleDrive(gauth)

InvalidConfigError: Invalid client secrets file ('Error opening file', 'client_secrets.json', 'No such file or directory', 2)

In [10]:
!pip install -q gdown

In [11]:
!zip -r /kaggle/working/training_results.zip /kaggle/working/runs/detect/train2

  adding: kaggle/working/runs/detect/train2/ (stored 0%)
  adding: kaggle/working/runs/detect/train2/train_batch0.jpg (deflated 5%)
  adding: kaggle/working/runs/detect/train2/args.yaml (deflated 53%)
  adding: kaggle/working/runs/detect/train2/labels.jpg (deflated 31%)
  adding: kaggle/working/runs/detect/train2/train_batch1.jpg (deflated 7%)
  adding: kaggle/working/runs/detect/train2/results.csv (deflated 64%)
  adding: kaggle/working/runs/detect/train2/weights/ (stored 0%)
  adding: kaggle/working/runs/detect/train2/weights/last.pt (deflated 8%)
  adding: kaggle/working/runs/detect/train2/weights/best.pt (deflated 8%)
  adding: kaggle/working/runs/detect/train2/train_batch2.jpg (deflated 6%)


In [15]:
!pip install -q gdrive

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 59.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 952.6/952.6 kB 55.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 3.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.8.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
pandas-gbq 0.29.1 requires google-api-core<3.0.0,>=2.10.2, but you have google-api-core 1.34.1 which is incompatible.
pandas-gbq 0.29.1 requires google-auth-oauthlib>=0.7.0, but you have google-auth-oauthlib 0.5.3 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
arviz 0.21.0 requires setuptools>=60.0.0, but you have setuptools 59.6.0 which is incompatible.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspe